In [36]:
TRAIN_PATH = 'data/original/train.jsonl'
DEV_PATH = 'data/original/dev.jsonl'
TEST_PATH = 'data/original/test.jsonl'

In [37]:
import pandas as pd
import json

def read_jsonl_to_dataframe(file_path):
    data = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                json_obj = json.loads(line)
                data.append(json_obj)
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON: {e}")

    df = pd.DataFrame(data)

    return df

In [38]:
df_train = read_jsonl_to_dataframe(TRAIN_PATH)
df_dev = read_jsonl_to_dataframe(DEV_PATH)
df_test = read_jsonl_to_dataframe(TEST_PATH)

# Processing data

In [39]:
from underthesea import word_tokenize
import regex as re

def tokenize(text):
    return word_tokenize(text, format="text")

def preprocess(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text) # remove punctuation
    text = re.sub(r"\s+", " ", text) # remove extra space
    text = text.strip()
    return text

def post_process(text):

    # text = tokenize(text)
    text = preprocess(text)

    return text

In [40]:
def get_all_span_aspect(text, label):
    
    for span in label:
        start, end, aspect_sentiment = span
        aspect = text[start:end]
        
        yield aspect, aspect_sentiment

df_train['all_span_aspect'] = df_train.apply(lambda row: list(get_all_span_aspect(row['text'], row['labels'])), axis=1)
df_test['all_span_aspect'] = df_test.apply(lambda row: list(get_all_span_aspect(row['text'], row['labels'])), axis=1)
df_dev['all_span_aspect'] = df_dev.apply(lambda row: list(get_all_span_aspect(row['text'], row['labels'])), axis=1)

In [41]:
df_train['all_span_aspect'].iloc[1]

[('Lag', 'PERFORMANCE#NEGATIVE'),
 ('hao pin', 'BATTERY#NEGATIVE'),
 ('Sam làm tệ quá, không bằng mấy con tàu cùng phân khúc',
  'GENERAL#NEGATIVE')]

In [42]:
# apply post process to all_span_aspect
df_train['all_span_aspect'] = df_train.apply(lambda row: [(post_process(aspect), sentiment) for aspect, sentiment in row['all_span_aspect']], axis=1)
df_test['all_span_aspect'] = df_test.apply(lambda row: [(post_process(aspect), sentiment) for aspect, sentiment in row['all_span_aspect']], axis=1)
df_dev['all_span_aspect'] = df_dev.apply(lambda row: [(post_process(aspect), sentiment) for aspect, sentiment in row['all_span_aspect']], axis=1)

In [43]:
df_train['all_span_aspect'].iloc[545]

[('xiaomi redmi note 9s tuyệt vời trên cả mong ợi', 'GENERAL#POSITIVE'),
 ('pin 5k max châu bò lun', 'BATTERY#POSITIVE'),
 ('cảm ơn các bạn nv dmx lộc bình cho hẳn 100 sao lun', 'SER&ACC#POSITIVE'),
 ('nên mua angs ồng tiền bỏ ra chỉ một từ tuyệt vời', 'GENERAL#POSITIVE')]

In [44]:
def process_data_frame(df):
    # new_df = pd.DataFrame()
    df['text'] = df['text'].apply(lambda x: post_process(x))
    # df['labels'] = df['labels']
    # df['all_span_aspect'] = df['all_span_aspect']
    return df

In [45]:
df_train = process_data_frame(df_train) # problem: process in here not same as process in above
df_dev = process_data_frame(df_dev)
df_test = process_data_frame(df_test)

In [46]:
df_train.text.iloc[545]

'xiaomi redmi note 9s tuyệt vời trên cả mong ợi tuyệt vời hơn xiaomi redmi note 8pro nhiều pin 5k max châu bò lun bạn nên mua angs ồng tiền bỏ ra chỉ một từ tuyệt vời cảm ơn các bạn nv dmx lộc bình cho hẳn 100 sao lun ms ón e nó về sáng nay'

In [47]:
# save data
import os

def save_data_to_jsonl(df, folder, filename):
    if not os.path.exists(folder):
        os.mkdir(folder)


    with open(os.path.join(folder, filename), 'w', encoding='utf-8') as file:

        # get all name of columns
        columns = df.columns
        for index, row in df.iterrows():
            json_obj = {}
            for column in columns:
                json_obj[column] = row[column]
            json.dump(json_obj, file, ensure_ascii=False)
            file.write('\n')

# save_data_to_csv(df_train, 'data/process_data', 'train.json')
# save_data_to_csv(df_dev, 'data/process_data', 'dev.json')
# save_data_to_csv(df_test, 'data/process_data', 'test.json')

# Combine data

In [1]:
# import pandas as pd
# import json

# def read_json_to_dataframe(filename):

#     with open(filename, 'r', encoding='utf-8') as file:
#         data = json.load(file)
    
#     df = pd.DataFrame(data)

#     return df

# df_train = read_json_to_dataframe('data/process_data/train.json')
# df_dev = read_json_to_dataframe('data/process_data/dev.json')
# df_test = read_json_to_dataframe('data/process_data/test.json')

In [48]:
def find_aspect(text, all_span_aspect):
    
    for span_aspect, sentiment in all_span_aspect:

        # find_text = span_aspect.replace('_', ' ')
        
        # find start and end of aspect in text
        start = text.find(span_aspect)
        end = start + len(span_aspect)

        # print(f"start: {start}, end: {end}, aspect: {span_aspect}, sentiment: {sentiment}")

        yield start, end, sentiment

df_train['labels'] = df_train.apply(lambda row: list(find_aspect(row['text'], row['all_span_aspect'])), axis=1)
df_dev['labels'] = df_dev.apply(lambda row: list(find_aspect(row['text'], row['all_span_aspect'])), axis=1)
df_test['labels'] = df_test.apply(lambda row: list(find_aspect(row['text'], row['all_span_aspect'])), axis=1)

In [49]:
df_train

,text,labels,all_span_aspect
0,pin sài tầm 50h cho pin 100 100 camera ổn tất ...,"[(0, 31, BATTERY#POSITIVE), (32, 41, CAMERA#PO...","[(pin sài tầm 50h cho pin 100 100, BATTERY#POS..."
1,lag và hao pin là cái tóm tắt về máy sa...,"[(0, 3, PERFORMANCE#NEGATIVE), (8, 15, BATTERY...","[(lag, PERFORMANCE#NEGATIVE), (hao pin, BATTER..."
2,tất cả đều ổn ngoại trừ lúc máy nóng lên thì p...,"[(78, 108, CAMERA#NEUTRAL), (109, 167, BATTERY...","[(cam chụp cũng gọi là tầm trung, CAMERA#NEUTR..."
3,ok mua máy ở tgdd chính sách đổi trả rất tốt r...,"[(18, 216, SER&ACC#POSITIVE)]",[(chính sách đổi trả rất tốt rất yên tâm khi m...
4,kiểu dáng thì đẹp cầm chắc tay nhưng loa nhỏ q...,"[(0, 30, DESIGN#POSITIVE), (37, 48, FEATURES#N...","[(kiểu dáng thì đẹp cầm chắc tay, DESIGN#POSIT..."
...,...,...,...
7780,máy mua đc 2thang bị lỗi camera trước có đổi đ...,"[(18, 37, CAMERA#NEGATIVE)]","[(bị lỗi camera trước, CAMERA#NEGATIVE)]"
7781,i bought this phone some hours ago because i l...,[],[]
7782,máy màn hình cứ tự sáng liên tục dù chẳng có t...,"[(0, 57, FEATURES#NEGATIVE)]",[(máy màn hình cứ tự sáng liên tục dù chẳng có...
7783,sau gần một tuần sử dụng máy cảm thấy máy dùng...,"[(25, 56, GENERAL#POSITIVE), (57, 123, FEATURE...","[(máy cảm thấy máy dùng rất thích, GENERAL#POS..."


In [54]:
del df_train['all_span_aspect']
del df_dev['all_span_aspect']
del df_test['all_span_aspect']

In [51]:
df_train.text = df_train.text.apply(lambda x: tokenize(x))
df_dev.text = df_dev.text.apply(lambda x: tokenize(x))
df_test.text = df_test.text.apply(lambda x: tokenize(x))

In [55]:
save_data_to_jsonl(df_train, 'data/process_data_split_words', 'train.jsonl')
save_data_to_jsonl(df_dev, 'data/process_data_split_words', 'dev.jsonl')
save_data_to_jsonl(df_test, 'data/process_data_split_words', 'test.jsonl')

In [ ]:
def print_data(index):
    print(f'Index: {index}')
    print(df_train.iloc[index].text)
    # print(df_train.iloc[index].labels)
    # print(df_train.iloc[index].all_span_aspect)
    print(df_train.iloc[index].labels)
    print("\n=============================================================================================\n")

# loop through all data
for i in range(len(df_train)):
    print_data(i)

# End